In [ ]:
import pymongo as py
from datetime import datetime, timedelta 
import pandas as pd
import numpy as np
import plotly

In [ ]:
parameters = ["date", "t_bett", "t_motor", "t_spindle", "DRZ2", "M8", "M121", "M127", "M7", "given2model", "welle_z"]
start_sec = datetime.strptime("2022-09-06T11:00:04.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end_sec = datetime.strptime("2022-09-07T02:00:59.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
timeshift = 2

In [ ]:
#####DB Config

host= "DevEdgeV32"
port=27017
collection= "modelLogs"

## Connect to data base and get the events

client = py.MongoClient(host= host, port=port)
db = client.h4ai
start= datetime.strptime("2022-09-06T00:00:31.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2022-09-07T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
event_list = db[collection].find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)


In [ ]:
# start_sec = datetime.strptime("2022-09-06T10:30:31.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# end_sec = datetime.strptime("2022-09-06T11:30:31.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# for event in event_list:
#     content = event["content"]
#     for data in content:
#         if data["date"] >= start_sec and data["date"] <= end_sec:
#             raw_data = data["raw_data"]
            # print("Date: " + str(data["date"]) + " t_bett: " + str(raw_data["t_bett"]) + " t_motor: " + str(raw_data["t_motor"]) 
            #       + " t_spindel: " + str(raw_data["t_spindle"]) + " DRZH: " + str(raw_data["DRZ2"]))


In [ ]:
df = pd.DataFrame(columns=parameters)

for event in event_list:
    content = event["content"]
    for data in content:
        if data["date"] >= start and data["date"] <= end:
            raw_data = data["raw_data"]

            dictRow = {'date': data["date"], 't_bett': raw_data["t_bett"], 't_motor': raw_data["t_motor"], 't_spindle': raw_data["t_spindle"], 'DRZ2': raw_data["DRZ2"],
                       'M8': raw_data["M8"], 'M121': raw_data["M121"], 'M127': raw_data["M127"], 'M7': raw_data["M7"], 'given2model': data["given2model"], 'welle_z':data["welle_z"]}
            df_row = pd.DataFrame([dictRow])
            df = pd.concat([df, df_row], axis=0, ignore_index=True)

print(df.head(5))

Create Plot

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio

scatter_mode= 'lines'

#y_axis_names= ['t_bett', 't_motor', 't_spindle', 'DRZ2', 'M8', 'M121', 'M127', 'M7', 'given2model', 'welle_z']
fig= make_subplots(rows=2, cols=2, shared_xaxes= True, print_grid= True, vertical_spacing=0.02)

fig.add_trace(go.Scatter(x= df['date'], y= df['t_bett'], name= 't_bett', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['t_motor'], name= 't_motor', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['t_spindle'], name= 't_spindle', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['welle_z'], name= 'welle_z', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'Abweichungen [mm]', row= 1, col= 1)

fig.add_trace(go.Scatter(x= df['date'], y= df['DRZ2'], name= 'DRZ2', mode= scatter_mode), row= 1, col= 2)
fig.update_yaxes(title_text= 'Drehzahl [1/min]', row= 1, col= 2)

fig.add_trace(go.Scatter(x= df['date'], y= df['M8'], name= 'M8', mode= scatter_mode), row= 2, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['M121'], name= 'M121', mode= scatter_mode), row= 2, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['M127'], name= 'M127', mode= scatter_mode), row= 2, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['M7'], name= 'M7', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'Kühlmittelbits', row= 2, col= 1)


fig.update_layout(height=600, width=1300, title_text="Daten im Dataframe")
fig.show()

Truncate to interesting region

In [ ]:
# Define important region

start_save = datetime.strptime("2022-09-06T08:07:04.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end_save = datetime.strptime("2022-09-06T20:29:59.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

In [ ]:
firstId = df.index[0]
lastId = df.index[-1]

idStartAll = df.index[df['date']>=start_save].tolist()
idStart = idStartAll[0]

idEndAll = df.index[df['date']>=end_save].tolist()
idEnd = idEndAll[0]

df = df[idStart:idEnd].copy()

# print(" firstID: " + str(idStart) + " StopId: " + str(idEnd))
# print(df.info())

In [ ]:
scatter_mode= 'lines'

#y_axis_names= ['t_bett', 't_motor', 't_spindle', 'DRZ2', 'M8', 'M121', 'M127', 'M7', 'given2model', 'welle_z']
fig= make_subplots(rows=2, cols=2, shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
#fig= make_subplots(rows=1, cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02, specs=[[{"secondary_y": True}]])

#for i in range(nrRows):
fig.add_trace(go.Scatter(x= df['date'], y= df['t_bett'], name= 't_bett', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['t_motor'], name= 't_motor', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['t_spindle'], name= 't_spindle', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['welle_z'], name= 'welle_z', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'Abweichungen [mm]', row= 1, col= 1)

fig.add_trace(go.Scatter(x= df['date'], y= df['DRZ2'], name= 'DRZ2', mode= scatter_mode), row= 1, col= 2)
fig.update_yaxes(title_text= 'Drehzahl [1/min]', row= 1, col= 2)

fig.add_trace(go.Scatter(x= df['date'], y= df['M8'], name= 'M8', mode= scatter_mode), row= 2, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['M121'], name= 'M121', mode= scatter_mode), row= 2, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['M127'], name= 'M127', mode= scatter_mode), row= 2, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y= df['M7'], name= 'M7', mode= scatter_mode), row= 2, col= 1)
fig.update_yaxes(title_text= 'Kühlmittelbits', row= 2, col= 1)


#fig.add_trace(go.Scatter(x= df['date'],y= predictions / 1000, name='Model prediction', mode= scatter_mode), row= len(visualization_columns) -1, col= 1)
fig.update_layout(height=600, width=1300, title_text="Daten im Dataframe")
fig.show()

From Dataframe to csv file

In [ ]:
df["date"] = df["date"] + timedelta(hours=timeshift)
df.to_csv("X:\\KI Praktikum\\5s_data_for_interpolation\\2022-09-06-cleaned.csv")
